In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import display, Markdown

import pandas as pd
import numpy as np

import json

np.__version__

/var/folders/b7/c67wpzpn25q0h072y9mmz_th0000gn/T/ipykernel_10853/2341679603.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


'1.26.3'

# Pull in data

In [6]:
PREVIOUS_OUTPUT_PATH = './data/22.2-OUTPUT-add-mjia-hand-labeling-and-relabel-uncategorized-using-them-as-exaples.csv'
prev_df = pd.read_csv(PREVIOUS_OUTPUT_PATH)
prev_df['song_id'] = prev_df['song'] + prev_df['performer']
prev_df

,performer,song,justification,love_song_sub_type,song_id
0,10cc,I'm Not In Love,"a repeated denial of being in love, which sugg...",It's Complicated,I'm Not In Love10cc
1,ABBA,Waterloo,"compares being in love with surrender, as if l...",Serenade,WaterlooABBA
2,AWB,Cut The Cake,symbolically about marriage but focuses on the...,NaN,Cut The CakeAWB
3,Aaliyah,Back & Forth,"a song about having a good time, but not a spe...",NaN,Back & ForthAaliyah
4,Aaron Neville,Everybody Plays The Fool,advising that everyone will be the fool in lov...,Longing & Heartbreak,Everybody Plays The FoolAaron Neville
...,...,...,...,...,...
5011,P!nk,Try,about the inevitability of getting hurt when l...,It's Complicated,TryP!nk
5012,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",Give Me Everything,"about seizing the moment, with a blend of part...",Courtship & Anticipation,"Give Me EverythingPitbull Featuring Ne-Yo, Afr..."
5013,Soko,We Might Be Dead By Tomorrow,emphasizing living in the current moment with ...,Longing & Heartbreak,We Might Be Dead By TomorrowSoko
5014,Taylor Swift,Blank Space,satirical view of the media's perception of he...,It's Complicated,Blank SpaceTaylor Swift


In [7]:
THRU_SEP_2023_PATH = './data/songs_df-through-Sep-2023.csv'
songs_df = pd.read_csv(THRU_SEP_2023_PATH)
songs_df

,song_id,chart_debut,song,performer,date_as_decimal,popularity_score,total_weeks_in_top_10
0,#9 DreamJohn Lennon,1975-02-15,#9 Dream,John Lennon,1975.13,3,2
1,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,2002-12-07,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,2002.93,56,11
2,'65 Love AffairPaul Davis,1982-04-24,'65 Love Affair,Paul Davis,1982.31,23,7
3,('til) I Kissed YouThe Everly Brothers,1959-09-07,('til) I Kissed You,The Everly Brothers,1959.68,44,7
4,(Can't Live Without Your) Love And AffectionNe...,1990-09-08,(Can't Live Without Your) Love And Affection,Nelson,1990.69,39,7
...,...,...,...,...,...,...,...
5136,amariJ. Cole,2021-05-29,amari,J. Cole,2021.41,6,1
5137,iSpyKYLE Featuring Lil Yachty,2017-04-01,iSpy,KYLE Featuring Lil Yachty,2017.25,31,8
5138,interludeJ. Cole,2021-05-22,interlude,J. Cole,2021.39,3,1
5139,"my.lifeJ. Cole, 21 Savage & Morray",2021-05-29,my.life,"J. Cole, 21 Savage & Morray",2021.41,9,1


# Pull out the data which needs labeling

June 18, 2024 note: Ok, so weirdly, MOST of the song that need labels (136) are from BEFORE 2023, and only 46 are from AFTER.

It seems to be rather small things, like "feat" vs "feature", or else "Save Your Tears" not mentioning it features Ariana in the later dataset.

I'm just gonna refetch all these, keep it simple, commit to the new dataset.

In [22]:
# get a df of songs where song_id is in songs_df but not in prev_df
new_songs_df = songs_df[~songs_df['song_id'].isin(prev_df['song_id'])]
new_songs_df = new_songs_df.sort_values(by='date_as_decimal')

len(new_songs_df[
    new_songs_df['date_as_decimal'] > 2022.999
])
new_songs_df[
    new_songs_df['date_as_decimal'] < 2022.999
]

new_songs_df

,song_id,chart_debut,song,performer,date_as_decimal,popularity_score,total_weeks_in_top_10
3000,Near YouRoger Williams,1958-09-29,Near You,Roger Williams,1958.74,3,3
3270,Peter GunnRay Anthony and His Orchestra,1959-02-23,Peter Gunn,Ray Anthony and His Orchestra,1959.15,5,3
4240,"The Happy OrganDave ""Baby"" Cortez",1959-04-27,The Happy Organ,"Dave ""Baby"" Cortez",1959.32,39,7
3386,Quiet VillageThe Exotic Sounds of Martin Denny,1959-05-18,Quiet Village,The Exotic Sounds of Martin Denny,1959.38,34,7
3187,Only YouFranck Pourcel's French Fiddles,1959-05-25,Only You,Franck Pourcel's French Fiddles,1959.40,6,4
...,...,...,...,...,...,...,...
3457,Rich Men North Of RichmondOliver Anthony Music,2023-08-26,Rich Men North Of Richmond,Oliver Anthony Music,2023.65,25,3
3222,Paint The Town RedDoja Cat,2023-09-02,Paint The Town Red,Doja Cat,2023.67,42,5
1955,I Remember EverythingZach Bryan Featuring Kace...,2023-09-09,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,2023.69,33,4
4641,Used To Be YoungMiley Cyrus,2023-09-09,Used To Be Young,Miley Cyrus,2023.69,3,1


# Fetch new labels using GPT4

In [23]:
import os
from openai import OpenAI

# First, activate the './.env' file which sets the OPENAI_API_KEY environment variable using python-dotenv
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [24]:
import ast

def eval_this_str(s):
    return ast.literal_eval(s)

In [25]:
LOVE_SONG_LABELING_PROMPT = """
You are an expert pop music critic, able to label pop songs as different kinds of love songs using subtle context, emotions, & relational implications.

For each Billboard Top 10 hit in the form [song, performer] in the larger input array, I want you to respond with a valid JSON array of:

[performer, song, justification, love_song_subtype_label]

The justification is a short snippet explaining why the song best fits in the chosen category.

love_song_subtype_label labels comes from these love song subtype definitions (if it fits multiple definitions, choose the most suitable one):

"Serenade": A song directed at someone the singer loves, expressing their deep affection, undying love, infatuation, or praising someone as a soulmate.

"Sexual Conquest": Songs that are less about the object of affection and more about the singer's own prowess or success in romantic pursuits.

"Love Song for the Self": Songs that express self-love and affirmation, whether for one's character or physical qualities, sometimes following a period of growth or learning from past relationships.

"Good Riddance": typically after a breakup with someone who now seems unsuitable. songs about having had someone's love, but being glad now you've moved on (or realizing you should), often involving self-empowerment, anger, revenge, relief, or bitterness, often because of betrayal or potentially cheating.

"Longing & Heartbreak": songs where you love or miss someone, but they are unable to return your love. Either because you broke up and the relationship is over, or because you were never together and the love was unrequited. COuld be full of regret, longing, frustration, sadness, hopelessness, even grief.

"Courtship & Anticipation": the speaker is courting or seeking someone's romantic or sexual love, but they don't have it yet. The love interest does not yet know the speaker well, but the speaker hopes they will want to get to know them better and or please them the way they want.

"It's Complicated": Songs about complicated relationships, often with conflicting emotions like love, hate, frustration, and desire intermingling, whether light-hearted or profound in tone.

"Platonic Love": Songs about non-romantic yet strong and meaningful relationships, such as friendships or familial bonds.

"NA": song is not a love song at all. This would include any non-love song, but also instrumentals, for example.

Here's an example input:

```json
[
	["'N Sync", "This I Promise You"],
	["John Legend", "All of Me"],
	["Beyonce", "Halo"],
	["T-Pain Featuring Lil Wayne", "Can't Believe It"],
	["Adele", "Someone Like You"],
	["Mariah Carey", "Without You"],
	["Dolly Parton", "Jolene"],
	["Bill Withers", "Ain't No Sunshine"],
	["The Rolling Stones", "Miss You"],
	["Whitney Houston", "I Will Always Love You"],
	["Taylor Swift", "Teardrops on My Guitar"],
	["Lesley Gore", "She's A Fool"],
	["Taylor Swift", "You Belong With Me"],
	["Justin Timberlake", "Suit & Tie"],
	["T-Pain Featuring Mike Jones", "I'm N Luv (Wit A Stripper)"],
	["Rihanna", "Only Girl (In the World)"],
	["Britney Spears", "Hold It Against Me"],
	["T-Pain", "Buy U A Drank (Shawty Snappin')"],
	["2Pac", "Dear Mama"],
	["Saweetie", "Best Friend (feat. Doja Cat)"],
	["James Taylor", "You've Got a Friend"],
	["Bill Withers", "Lean On Me"],
	["Little Mix and Jason Derulo", "Secret Love Song"],
	["Katy Perry", "Hot N Cold"],
	["Pat Benatar", "Love Is a Battlefield"],
	["Lauryn Hill", "Ex-Factor"],
	["24kGoldn", "Mood"],
	["T-Pain", "I'm Sprung"],
	["Kelly Clarkson", "Since U Been Gone"],
	["Blu Cantrell", "Hit 'Em Up Style (Oops!)"],
	["Selena Gomez", "Look at Her Now"],
	["Taylor Swift", "Picture To Burn"],
	["Christina Aguilera", "Fighter"],
	["Jamie Foxx Featuring T-Pain", "Blame It"],
	["6ix9ine Featuring Nicki Minaj & Murda Beatz", "FEFE"],
	["The Beach Boys", "I Get Around"],
	["Robin Thicke ft. Pharrell & T.I.", "Blurred Lines"],
	["112", "Peaches & Cream"],
	["Lady Gaga", "Poker Face"],
	["Ariana Grande", "Thank U, Next"],
	["Lizzo", "Good As Hell"],
	["Christina Aguilera", "Beautiful"],
	["Britney Spears", "Womanizer"],
	["TLC", "No Scrubs"],
	["Meghan Trainor", "NO"],
	["Radiohead", "Creep"],
	["Rick Springfield", "Jessie's Girl"],
	["21 Savage & Metro Boomin", "Runnin"],
	["twenty one pilots", "Heathens"],
	["will.i.am & Britney Spears", "Scream & Shout"],
    ["3OH!3 Featuring Ke$ha","My First Kiss"],["Anna Kendrick","Cups (Pitch Perfect\'s When I\'m Gone)"],["Ava Max","Sweet But Psycho"],["Bruno Mars & Cardi B","Finesse"],["Calvin Harris","Summer"],["Calvin Harris Featuring Rihanna","This Is What You Came For"],["Camila Cabello Featuring Young Thug","Havana"],["Capital Cities","Safe And Sound"],["Chris Brown","Don\'t Wake Me Up"],["Cobra Starship Featuring Sabi","You Make Me Feel..."],["Elton John & Dua Lipa","Cold Heart (PNAU Remix)"],["Fetty Wap","Trap Queen"],["Harry Styles","As It Was"],["Hozier","Take Me To Church"],["Katy Perry Featuring Kanye West","E.T."],["Kendrick Lamar & SZA","All The Stars"],["Lil Nas X","Panini"],["Lil Wayne","How To Love"],["Lil Wayne Featuring Kendrick Lamar","Mona Lisa"],["Michael Jackson & Justin Timberlake","Love Never Felt So Good"],["Owl City","Fireflies"],["P!nk","Try"],["Pitbull Featuring Ne-Yo, Afrojack & Nayer","Give Me Everything"],["Soko","We Might Be Dead By Tomorrow"],["Taylor Swift","Blank Space"],["The Chainsmokers","Paris"]

]
```

Here's the sample corresponding output:

```json
{"output":[
    ["'N Sync","This I Promise You","promising to always love someone","Serenade"],["John Legend","All of Me","praises lover, who's affection makes any struggle worth it, promises to give all of them","Serenade"],["Beyonce","Halo","celebrates the angelic quality of a lover and testifies to the positive changes as a result of this love","Serenade"],["T-Pain Featuring Lil Wayne","Can't Believe It","praises love interest, telling her of all the things he'll do for her","Serenade"],["Adele","Someone Like You","bemoans that a lover has moved on to a happy life with another partner while the speaker struggles to move on","Longing & Heartbreak"],["Mariah Carey","Without You","at the moment of a breakup, she expresses despair over everything she will lose","Longing & Heartbreak"],["Dolly Parton","Jolene","her heart is breaking because she fears her male lover will be easily stolen by another woman","Longing & Heartbreak"],["Bill Withers","Ain't No Sunshine","when a lover is away, he expresses that his world seems to become darker","Longing & Heartbreak"],["The Rolling Stones","Miss You","expresses how deeply he misses the lover, and wishes he didn't have to wait","Longing & Heartbreak"],["Whitney Houston","I Will Always Love You","wishes her now gone lover well, reminisces, will never forget","Longing & Heartbreak"],["Taylor Swift","Teardrops on My Guitar","she's in love with a friend who's in love with someone else, she has to pretend she doesn't love him but secretly she's suffering","Longing & Heartbreak"],["Lesley Gore","She's A Fool","her love is with another woman who doesn't appreciate his virtues, longs to some day be noticed by him","Longing & Heartbreak"],["Taylor Swift","You Belong With Me","she's making an empassioned argument that her love object should be with her instead of who he's currently with","Courtship & Anticipation"],["Justin Timberlake","Suit & Tie","looking forward to a night of dancing with his beloved","Courtship & Anticipation"],["T-Pain Featuring Mike Jones","I'm N Luv (Wit A Stripper)","he has strong romantic and sexual feelings for a stripper who doesn't yet know him personally","Courtship & Anticipation"],["Rihanna","Only Girl (In the World)","telling her new love how she wants him to make her feel","Courtship & Anticipation"],["Britney Spears","Hold It Against Me","she's interested in a stragner and approaches them hoping to see if they might get together","Courtship & Anticipation"],["T-Pain","Buy U A Drank (Shawty Snappin')","he spots a woman at the bar and offers to buy her a drink as a prelude to potentially getting together","Courtship & Anticipation"],["2Pac","Dear Mama","love and appreciation for his mother","Platonic Love"],["Saweetie","Best Friend (feat. Doja Cat)","boasts about a close friend who is succesful and strong","Platonic Love"],["James Taylor","You've Got a Friend","promises a close friend they'll be there thick and thin","Platonic Love"],["Bill Withers","Lean On Me","invites a friend to rely on him when they are struggling, as everyone needs platonic support at times","Platonic Love"],["Little Mix and Jason Derulo","Secret Love Song","the speakers long to be united with their lover in public, but must remain in secret","It's Complicated"],["Katy Perry","Hot N Cold","about navigating a lover who cannot make up their mind and is inconsistent","It's Complicated"],["Pat Benatar","Love Is a Battlefield","the relationship she is in alternates between being loving and painful and she doesn't feel like she can leave","It's Complicated"],["Lauryn Hill","Ex-Factor","she feels trapped in a relationship where she feels she's giving more than she's recieving, but whenever she tries to leave, she can't","It's Complicated"],["24kGoldn","Mood","bemoans why his female love interest is always difficult, moody, and inconsistent","It's Complicated"],["T-Pain","I'm Sprung","knows his love interest doesn't deserve him, yet he can't stay away, he vascilates between feelings of deep love and wanting to get away as fast as possible","It's Complicated"],["Kelly Clarkson","Since U Been Gone","ending a bad relationship leaves her feeling better and more in control","Good Riddance"],["Blu Cantrell","Hit 'Em Up Style (Oops!)","reacts to being cheated on by gleefully spending her ex-lover's money, as an act or revents","Good Riddance"],["Selena Gomez","Look at Her Now","after being cheated on, she has now moved on and is better than ever","Good Riddance"],["Taylor Swift","Picture To Burn","after realizing how self-centered and unsuitable her ex-lover is, she vents about all of his negative qualities","Good Riddance"],["Christina Aguilera","Fighter","she thanks the lover because his abuse actually ended up making her stronger and wiser","Good Riddance"],["Jamie Foxx Featuring T-Pain","Blame It","speaker brags about his sexual power & alure, confident his love interest will want to hook up with him","Sexual Conquest"],["6ix9ine Featuring Nicki Minaj & Murda Beatz","FEFE","about sexual prowess with many sexual references","Sexual Conquest"],["The Beach Boys","I Get Around","brag about constantly seeking out new women and never being rejected","Sexual Conquest"],["Robin Thicke ft. Pharrell & T.I.","Blurred Lines","speaker brags about being about to 'get nasty' with a good girl who he claims wants sex","Sexual Conquest"],["112","Peaches & Cream","about sex & being addicted to a tasty woman","Sexual Conquest"],["Lady Gaga","Poker Face","brags about a male sex partner who does not know she is sleeping with a women as well","Sexual Conquest"],["Ariana Grande","Thank U, Next","brags about being unfased and even improved by past relationships ending, and in fact is now thriving in life","Love Song for the Self"],["Lizzo","Good As Hell","an anthem for leaving behidn negativity or men who don't love you anymore, and instead enjoying who you are with pride","Love Song for the Self"],["Christina Aguilera","Beautiful","an anthem proclaiming ones beauty, even in the face of deragatory words or spiteful ex-lovers","Love Song for the Self"],["Britney Spears","Womanizer","in her prowess, she recognizes a womanizer and rejects his advances","Love Song for the Self"],["TLC","No Scrubs","speakers asserts that they will reject so-called scrubs because she wants a higher quality man","Love Song for the Self"],["Meghan Trainor","NO","an anthem for women to confidently reject unwanted male advances, because the men aren't needed for the women to be in the zone","Love Song for the Self"],["Radiohead","Creep","in contrast to his love interest's angelic nature, he feels ugly and invisible, and hates on himself, which does not fit in Longing & Hearbreak or even Good Riddance","Longing & Heartbreak"],["Rick Springfield","Jessie's Girl","praises his friends lover and expresses he's confused she isn't into him too, and he's jealous: themes to unusual to fit into the given love song subtypes.","It's Complicated"],["21 Savage & Metro Boomin","Runnin","a gangster rap, not romance","NA"],["twenty one pilots","Heathens","about social outcasts, not romance","NA"],["will.i.am & Britney Spears","Scream & Shout","about dancing in the club, with no real elements of romance directed toward a specific love interest","NA"],
    ["3OH!3 Featuring Ke$ha","My First Kiss","reminiscing about a first kiss, which is a fond memory but not necessarily a song professing love","Courtship & Anticipation"],["Anna Kendrick","Cups (Pitch Perfect\'s When I\'m Gone)","a song about leaving and moving on to pursue one\'s dreams, ambiguous enough that it doesn\'t quite capture a specific romantic mood","Longing & Heartbreak"],["Ava Max","Sweet But Psycho","portrays a woman in love who is misunderstood to be \'psycho\'","It\'s Complicated"],["Bruno Mars & Cardi B","Finesse","a song celebrating mutual attraction and good times, though not directly focused on love or relationship depth","Love Song for the Self"],["Calvin Harris","Summer","captures a summertime feeling of freedom and excitement, but not expressly a love song","It\'s Complicated"],["Calvin Harris Featuring Rihanna","This Is What You Came For","a party song about an electric connection with someone, possibly romantic, but undefined","Courtship & Antisipation"],["Camila Cabello Featuring Young Thug","Havana","a narrative about being romantically entangled with a suave character named \'East Atlanta\', though not deeply emotional","Longing & Heartbreak"],["Capital Cities","Safe And Sound","an upbeat melody offering support and security, can be interpreted as platonic or romantic","Serenade"],["Chris Brown","Don\'t Wake Me Up","talks about a dream of a perfect relationship he doesn\'t want to end","Serenade"],["Cobra Starship Featuring Sabi","You Make Me Feel...","about the exhilarating feeling someone gives you, could be interpreted as a romantic song","Courtship & Anticipation"],["Elton John & Dua Lipa","Cold Heart (PNAU Remix)","mixes lyrics from previous Elton John\'s songs, dealing with longing for warmth in a cold-hearted world","NA"],["Fetty Wap","Trap Queen","about a woman helping her man in his drug business, love within a criminal context","Serenade"],["Harry Styles","As It Was","reflects on changes and how someone fits into the singer\'s life","NA"],["Hozier","Take Me To Church","metaphorically compares loving a person to religion, and critiques institutions that oppose his love","Serenade"],["Katy Perry Featuring Kanye West","E.T.","compares a lover to an alien, otherworldly love interest","Courtship & Anticipation"],["Kendrick Lamar & SZA","All The Stars","mentions a relationship but in the context of larger themes of life and legacy","NA"],["Lil Nas X","Panini","about fans\' attachment and his own personal journey, not directly a love song","NA"],["Lil Wayne","How To Love","a narrative about a woman\\u2019s life and her struggle to understand love","Courtship & Anticipation"],["Lil Wayne Featuring Kendrick Lamar","Mona Lisa","describes using women to set up a robbery, trust and love are manipulated for personal gain, which doesn\\u2019t fit cleanly into defined love song subcategories","Good Riddance"],["Michael Jackson & Justin Timberlake","Love Never Felt So Good","remembers when love was good and pure, sort of a celebration of past love","Serenade"],["Owl City","Fireflies","about wonder at the world more than a love song, though could suggest a sense of platonic connection","NA"],["P!nk","Try","about the inevitability of getting hurt when looking for love but encourages to try again","It\'s Complicated"],["Pitbull Featuring Ne-Yo, Afrojack & Nayer","Give Me Everything","about seizing the moment, with a blend of party and romantic elements","Courtship & Anticipation"],["Soko","We Might Be Dead By Tomorrow","emphasizing living in the current moment with the person they care about, has a romantic undertone","Longing & Heartbreak"],["Taylor Swift","Blank Space","satirical view of the media\'s perception of her dating life, embodying a character","It\'s Complicated"],["The Chainsmokers","Paris","reflects on an escape with a partner, hinting at a deeper connection but not overtly romantic","Serenade"]
    ]}
```

Ok, now generate the output for this input:


"""

In [26]:
# pretty print the JSON string
# display(Markdown(LOVE_SONG_LABELING_PROMPT))
# LOVE_SONG_LABELING_PROMPT

In [48]:
def get_love_song_labels_for_rows(row_subset_str):
    try: 
        completion = client.chat.completions.create(
          model="gpt-4o",
          response_format={'type': "json_object"},
          messages=[
            {"role": "user", "content": LOVE_SONG_LABELING_PROMPT},
            {"role": "user", "content": row_subset_str},
          ]
        )
        content = completion.choices[0].message.content
        # return content
        response_content = eval_this_str(content)['output']
        # print(f"Response contains {completion['usage']['completion_tokens']} tokens out of 4096 maximum allowed tokens.")
        return response_content
    except Exception as e:
        print('ERROR FETCHING', e)
        return []

### Fetch in batches

In [49]:

def fetch_all_data_in_batches(df, increment=25):
    gpt_labeled_result = []
    for i in range(101):
        subset = df[['performer', 'song']][i*increment : (i+1)*increment]
        if len(subset):
            subset_as_json_string = subset.to_json(orient='values')
            print([i*increment, (i+1)*increment])
            print(subset_as_json_string, '\n')
            fetched_and_formatted_res = get_love_song_labels_for_rows(subset_as_json_string)
            print(fetched_and_formatted_res)
            gpt_labeled_result = [*gpt_labeled_result, *fetched_and_formatted_res]
    return gpt_labeled_result

In [50]:
def get_file_path(postfix):
    return f'./data/22-TEMP-gpt4-re-relabel-love-song-types-{postfix}.csv'

In [51]:
def save_to_temp_csv(gpt_labeled_result, run_num):
    first_label_pass_df = pd.DataFrame(
        gpt_labeled_result, 
        columns=['performer', 'song', 
                 'justification',
                 'love_song_sub_type']
    )
    first_label_pass_df.to_csv(get_file_path(run_num), index=False)

# (run a test call to be sure things look ok)
For 100 rows at once:
For GPT4 turbo: 3min, about 3400 tokens out of 4096 maximum allowed tokens 

In [52]:
%%time
TEST_POSTFIX = 'JUST A TEST, DELETE ME'
save_to_temp_csv(
    fetch_all_data_in_batches(
        new_songs_df.iloc[0:3]
    ), 
    TEST_POSTFIX
)

[0, 25]
[["Roger Williams","Near You"],["Ray Anthony and His Orchestra","Peter Gunn"],["Dave \"Baby\" Cortez","The Happy Organ"]] 

[['Roger Williams', 'Near You', 'a tender instrumental, generating a sense of romantic longing even without lyrics', 'Longing & Heartbreak'], ['Ray Anthony and His Orchestra', 'Peter Gunn', 'an instrumental theme song for a TV show, not a love song', 'NA'], ['Dave "Baby" Cortez', 'The Happy Organ', 'an instrumental showcasing joy and energy but not romantic themes', 'NA']]
CPU times: user 31.2 ms, sys: 3.14 ms, total: 34.3 ms
Wall time: 2.27 s


In [37]:
pd.read_csv(get_file_path(TEST_POSTFIX))#.iloc[1].justification

,performer,song,justification,love_song_sub_type
0,Roger Williams,Near You,"instrumental cover, so not applicable as a lov...",NaN
1,Ray Anthony and His Orchestra,Peter Gunn,"instrumental theme song, so not applicable as ...",NaN
2,"Dave ""Baby"" Cortez",The Happy Organ,"instrumental piece, so not applicable as a lov...",NaN


# Ok, run the actual fetch

In [53]:
FILE_KEY = 'needed-updates-thru-sep-2023-data'
save_to_temp_csv(
    fetch_all_data_in_batches(
        new_songs_df.iloc[0:1000],
        25
    ), 
    FILE_KEY
)

[0, 25]
[["Roger Williams","Near You"],["Ray Anthony and His Orchestra","Peter Gunn"],["Dave \"Baby\" Cortez","The Happy Organ"],["The Exotic Sounds of Martin Denny","Quiet Village"],["Franck Pourcel's French Fiddles","Only You"],["Santo & Johnny","Sleep Walk"],["Johnny And The Hurricanes","Red River Rock"],["Sandy Nelson","Teen Beat"],["Percy Faith And His Orchestra","The Theme From \"A Summer Place\""],["Bill Black's Combo","White Silver Sands"],["The Ventures","Walk -- Don't Run"],["Floyd Cramer","Last Date"],["Ferrante & Teicher","Exodus"],["Floyd Cramer","On The Rebound"],["Mar-Keys","Last Night"],["Sandy Nelson","Let There Be Drums"],["Ferrante & Teicher","Tonight"],["Kenny Ball and his Jazzmen","Midnight In Moscow"],["Billy Joe & The Checkmates","Percolator (Twist)"],["Mr. Acker Bilk","Stranger On The Shore"],["Ray Stevens","Ahab, The Arab"],["Booker T. & The MG's","Green Onions"],["Baby Cortez","Rinky Dink"],["Herb Alpert And Tijuana Brass","The Lonely Bull (El Solo Torro)"],["

# Process results

In [54]:
love_song_subtypes_df = pd.concat([
    pd.read_csv(get_file_path(FILE_KEY)),
])
love_song_subtypes_df = love_song_subtypes_df.drop_duplicates(subset=['performer', 'song'])
love_song_subtypes_df

,performer,song,justification,love_song_sub_type
0,Roger Williams,Near You,"This is an instrumental track, so it does not ...",NaN
1,Ray Anthony and His Orchestra,Peter Gunn,This is an instrumental piece related to a det...,NaN
2,"Dave ""Baby"" Cortez",The Happy Organ,This track is an instrumental featuring organ ...,NaN
3,The Exotic Sounds of Martin Denny,Quiet Village,This is an instrumental piece meant to evoke a...,NaN
4,Franck Pourcel's French Fiddles,Only You,Though many versions of 'Only You' are romanti...,NaN
...,...,...,...,...
177,Oliver Anthony Music,Rich Men North Of Richmond,the song deals with social commentary and pers...,NaN
178,Doja Cat,Paint The Town Red,"about confidence and self-empowerment, express...",Love Song for the Self
179,Zach Bryan Featuring Kacey Musgraves,I Remember Everything,reflects on a past relationship with mixed emo...,Longing & Heartbreak
180,Miley Cyrus,Used To Be Young,"reminiscing about past experiences and growth,...",Love Song for the Self


In [56]:
# replace "Platonic Love" with np.nan since we're ignoring this category
love_song_subtypes_df['love_song_sub_type'] = love_song_subtypes_df['love_song_sub_type'].replace('Platonic Love', np.nan)

In [57]:
love_song_subtypes_df.love_song_sub_type.value_counts()

love_song_sub_type
Serenade                    24
Longing & Heartbreak        16
It's Complicated            15
Love Song for the Self      11
Courtship & Anticipation     7
Good Riddance                6
Name: count, dtype: int64

# Merge a complete dataset using old & new fetched metadata

In [60]:
# Concat prev & new fetched results
NEEDED_COLS = ['performer', 'song', 'justification', 'love_song_sub_type']
prev_metadata_df = prev_df[NEEDED_COLS]
metadata_source = pd.concat([
    prev_metadata_df,
    love_song_subtypes_df
])

# Merge the metadata with the songs_df, but only keep the rows from songs_df (discard any that are only in metadata_source)
merged_df = pd.merge(
    songs_df,
    metadata_source,
    on=['performer', 'song'],
    how='left'
)

assert(len(merged_df) == len(songs_df))

merged_df = merged_df.drop(columns=['justification'])
merged_df

,song_id,chart_debut,song,performer,date_as_decimal,popularity_score,total_weeks_in_top_10,love_song_sub_type
0,#9 DreamJohn Lennon,1975-02-15,#9 Dream,John Lennon,1975.13,3,2,NaN
1,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,2002-12-07,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,2002.93,56,11,It's Complicated
2,'65 Love AffairPaul Davis,1982-04-24,'65 Love Affair,Paul Davis,1982.31,23,7,NaN
3,('til) I Kissed YouThe Everly Brothers,1959-09-07,('til) I Kissed You,The Everly Brothers,1959.68,44,7,Serenade
4,(Can't Live Without Your) Love And AffectionNe...,1990-09-08,(Can't Live Without Your) Love And Affection,Nelson,1990.69,39,7,Longing & Heartbreak
...,...,...,...,...,...,...,...,...
5136,amariJ. Cole,2021-05-29,amari,J. Cole,2021.41,6,1,NaN
5137,iSpyKYLE Featuring Lil Yachty,2017-04-01,iSpy,KYLE Featuring Lil Yachty,2017.25,31,8,Sexual Conquest
5138,interludeJ. Cole,2021-05-22,interlude,J. Cole,2021.39,3,1,NaN
5139,"my.lifeJ. Cole, 21 Savage & Morray",2021-05-29,my.life,"J. Cole, 21 Savage & Morray",2021.41,9,1,NaN


# export

In [61]:
WITH_LOVE_SONG_TYPES_PATH = './data/22.2-v2-OUTPUT-data-through-sep-2023-ALL-SONGS-LABELLED-WITH-FINAL-VERSION.csv'
merged_df.to_csv(WITH_LOVE_SONG_TYPES_PATH, index=False)